# Ayudantía (Soluciones)

In [12]:
import pandas as pd
from datetime import datetime

## Solución 1 — Normalización de nombres y correos

In [13]:
df1 = pd.DataFrame({
    'id': ['01',' 02 ','003'],
    'nombre_completo': [' ana GÓMEZ  ', 'LUIS   pérez', 'soFÍA   díAz '],
    'emails': ['ana@uc.cl; ANA@GMAIL.COM', 'l.perez@uc.cl', 'sdiaz@mail.com; sofia@uc.cl']
})

df1

,id,nombre_completo,emails
0,01,ana GÓMEZ,ana@uc.cl; ANA@GMAIL.COM
1,02,LUIS pérez,l.perez@uc.cl
2,003,soFÍA díAz,sdiaz@mail.com; sofia@uc.cl


In [14]:
res1 = df1.copy()

## Normalización nombre
res1['nombre_completo'] = (res1['nombre_completo']
                           .str.strip()
                           .str.title()
                           .str.replace(r'\s+', ' ', regex=True))

## Explode de emails
res1['emails'] = res1['emails'].str.lower().str.replace(' ', '')
res1['emails'] = res1['emails'].str.split(';')
res1 = res1.explode('emails', ignore_index=True)

## Normalización id
res1['id'] = (res1['id'].str.strip()
                        .astype('Int64'))

res1

,id,nombre_completo,emails
0,1,Ana Gómez,ana@uc.cl
1,1,Ana Gómez,ana@gmail.com
2,2,Luis Pérez,l.perez@uc.cl
3,3,Sofía Díaz,sdiaz@mail.com
4,3,Sofía Díaz,sofia@uc.cl


## Solución 2 — Limpieza de montos

In [15]:
df2 = pd.DataFrame({'monto': ['10.000', '12.500', '3.000', '4.500']})
df2

,monto
0,10.000
1,12.500
2,3.000
3,4.500


In [16]:
def miles_a_float(s):
    # En este set consideramos '.' como miles; no hay decimales explícitos, los tratamos como enteros
    s = s.replace('.', '')  # quitar separadores de miles
    try:
        return float(s)
    except:
        return None

res2 = df2.copy()
res2['monto_float'] = res2['monto'].apply(miles_a_float)
res2

,monto,monto_float
0,10.000,10000.0
1,12.500,12500.0
2,3.000,3000.0
3,4.500,4500.0


## Solución 3 — Fechas en mismo formato

In [17]:
df3 = pd.DataFrame({'fecha': ['12/03/2024','10/05/2024','15/06/2024','01/12/2024']})
df3

,fecha
0,12/03/2024
1,10/05/2024
2,15/06/2024
3,01/12/2024


In [ ]:
res3 = df3.copy()
res3['fecha'] = pd.to_datetime(res3['fecha'], format='%d/%m/%Y', errors='coerce')
res3['fecha'] = res3['fecha'].dt.strftime('%Y-%m-%d')
res3['year'] = pd.to_datetime(res3['fecha']).dt.year
res3['month-day'] = pd.to_datetime(res3['fecha']).dt.strftime('%m-%d')
res3

,fecha,year,month-day
0,2024-03-12,2024,03-12
1,2024-05-10,2024,05-10
2,2024-06-15,2024,06-15
3,2024-12-01,2024,12-01


## Solución 4 — Extracción con `str.split` y `explode`

In [19]:
df4 = pd.DataFrame({
    'id': [1,2,3],
    'region_comuna': ['RM|Santiago', 'V|Viña del Mar', 'VIII|Concepción'],
    'hobbies': ['futbol;leer', 'cine', 'musica;ajedrez;gaming']
})
df4

,id,region_comuna,hobbies
0,1,RM|Santiago,futbol;leer
1,2,V|Viña del Mar,cine
2,3,VIII|Concepción,musica;ajedrez;gaming


In [20]:
res4 = df4.copy()
res4[['region','comuna']] = res4['region_comuna'].str.split('|', expand=True)
res4['hobbies'] = res4['hobbies'].str.split(';')
res4 = res4.explode('hobbies', ignore_index=True)
res4 = res4[['id','region','comuna','hobbies']]
res4

,id,region,comuna,hobbies
0,1,RM,Santiago,futbol
1,1,RM,Santiago,leer
2,2,V,Viña del Mar,cine
3,3,VIII,Concepción,musica
4,3,VIII,Concepción,ajedrez
5,3,VIII,Concepción,gaming


## Solución 5 — Conexión de DataFrames y cardinalidad

In [21]:
alumnos = pd.DataFrame({'id':[1,2,3], 'nombre':['Ana','Luis','Sofía']})
notas   = pd.DataFrame({'id':[1,2,4], 'nota':[6.5,5.8,7.0]})
emails  = pd.DataFrame({'id':[1,1,2], 'email':['ana@a.com','ana@b.com','luis@c.com']})

print('ALUMNOS:')
display(alumnos)
print('\nNOTAS:')
display(notas)
print('\nEMAILS:')
display(emails)


ALUMNOS:


,id,nombre
0,1,Ana
1,2,Luis
2,3,Sofía



NOTAS:


,id,nota
0,1,6.5
1,2,5.8
2,4,7.0



EMAILS:


,id,email
0,1,ana@a.com
1,1,ana@b.com
2,2,luis@c.com


In [22]:
res5_inner = pd.merge(alumnos, notas, on='id', how='inner')
res5_left = pd.merge(alumnos, emails, on='id', how='left')
print('INNER:')
display(res5_inner)
print('\nLEFT:')
display(res5_left)

INNER:


,id,nombre,nota
0,1,Ana,6.5
1,2,Luis,5.8



LEFT:


,id,nombre,email
0,1,Ana,ana@a.com
1,1,Ana,ana@b.com
2,2,Luis,luis@c.com
3,3,Sofía,NaN


En el LEFT aparecen filas duplicadas para Ana (id=1) porque existe una relación 1 a N entre `alumnos` (id único) y `emails` (varios correos por id). Cada email genera una fila.